This script will be for linking the siteIDs to the catchment IDs - which are either NHDPlusIDs,
gridcodes, or new catIDs made from the gridcodes for BB. This will give us a way to link the
thermal sensitivity responses by year to the covariates. This script also includes code to
check the watersheds. The watersheds are individual feature classes to avoid overlapping
polygons so the best way to check they are correct is to 1) check that every siteID/catID
has a watershed, and 2) check that the watershed area approximately matches the watershed
area from the flow accumulation grid. They won't be exact for sites that are not close to
the catchment outlet and could be quite different for very small watersheds in some cases.

# Watershed Summaries

1. read in all watersheds feature classes
2. create a table with the NHDPlusID/catID of the watershed name, region, and watershed area
3. merge original point feature classes (inside and outside bb)
4. do a spatial join with merged catchments by region to get catID and region on the points dataset
5. clip to the region and save in each regional gdb
6. merge all point feature classes with siteIDs and catIDs into one table
7. join the watershed area from the watershed fcs to the points table using the catID
8. join the watershed area from the fac grid to the points table using the catID
9. compare the results to ensure that watersheds have been created correctly

In [4]:
# steps 1 and 2

import arcpy
import os
import pandas as pd

# regions = ["Cook_Inlet"] # still troubleshooting large watersheds for this region.
# Bristol_Bay - need to rerun for this region
regions = ["Kodiak", "Copper_River", "Prince_William_Sound"]

for region in regions:
    arcpy.env.workspace = "W:\\GIS\\AKSSF\\" + region + "\\" + region + ".gdb\\Watersheds"
    wtds = arcpy.ListFeatureClasses()
    print(wtds)
    print(len(wtds))
    wtdList = []

    for wtd in wtds:
        wtdName = wtd[4:20]
        print("Starting wtd: " + wtdName)
        wtdPath = os.path.join(arcpy.env.workspace, wtd)
        lstFields = arcpy.ListFields(wtdPath)
        if "Area_km2" in lstFields:
            print("Area already calculated")
        else:
            arcpy.AddField_management(wtdPath, "Area_km2", "DOUBLE")
            expression1 = "{0}".format("!SHAPE.area@SQUAREKILOMETERS!")
            arcpy.CalculateField_management(wtdPath, "Area_km2", expression1, "PYTHON", )
        wtdArea = [row[0] for row in arcpy.da.SearchCursor(wtdPath, ['Area_km2'])]
        print("wtdName: " + str(wtdArea))
        wtdList.append({'Region': region, 'catID': wtdName, 'Area_km2': wtdArea})

wtdDf = pd.DataFrame(wtdList)
print(wtdDf)

['wtd_48267', 'wtd_49617', 'wtd_50197', 'wtd_64593', 'wtd_72144', 'wtd_76954', 'wtd_77794', 'wtd_90346', 'wtd_93176', 'wtd_94216', 'wtd_97276', 'wtd_99516', 'wtd_100826', 'wtd_101556', 'wtd_103096', 'wtd_103196', 'wtd_103296', 'wtd_103456', 'wtd_103496', 'wtd_106626', 'wtd_106676', 'wtd_107796', 'wtd_107816', 'wtd_108356', 'wtd_128685', 'wtd_129955', 'wtd_130295', 'wtd_130735']
28
Starting wtd: 48267
wtdName: [80.9867]
Starting wtd: 49617
wtdName: [294.5615]
Starting wtd: 50197
wtdName: [49.5008]
Starting wtd: 64593
wtdName: [65.9962]
Starting wtd: 72144
wtdName: [4.6374]
Starting wtd: 76954
wtdName: [31.3052]
Starting wtd: 77794
wtdName: [55.2271]
Starting wtd: 90346
wtdName: [7.9572]
Starting wtd: 93176
wtdName: [11.0158]
Starting wtd: 94216
wtdName: [9.9504]
Starting wtd: 97276
wtdName: [25.5897]
Starting wtd: 99516
wtdName: [15.5985]
Starting wtd: 100826
wtdName: [27.3517]
Starting wtd: 101556
wtdName: [25.7735]
Starting wtd: 103096
wtdName: [53.279]
Starting wtd: 103196
wtdName: [

# step 3 - done

import arcpy

gdb = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb"
arcpy.env.workspace = gdb
bb_pts = gdb + "\\bb_md_verified_DM"
other_pts = gdb + "\\sites_outside_bb_verified_DM"

output = "akssf_pts_verified"
arcpy.Merge_management([bb_pts, other_pts], output)

In [23]:
# steps 4-6

import arcpy
import os
import numpy
import pandas as pd

arcpy.env.overwriteOutput = True
regions = ["Kodiak", "Copper_River", "Prince_William_Sound"]
# regions = ["Cook_Inlet", "Bristol_Bay"]
points = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\akssf_pts_verified"
sites_lst = []

for region in regions:
    arcpy.env.workspace = "W:\\GIS\\AKSSF\\" + region + "\\" + region + ".gdb"
    feat_lst = arcpy.ListFeatureClasses()
    print(feat_lst)
    if("sites_sj" in feat_lst):
        print("Feature Class exists")
    else:
        cats = os.path.join(arcpy.env.workspace, "cats_merge")
        arcpy.Clip_analysis(points, cats, "sites_clip")
        arcpy.SpatialJoin_analysis("sites_clip", cats, "sites_sj")
        arcpy.DeleteFeatures_management("sites_clip")
        arcpy.AddField_management("sites_sj", "catID", "DOUBLE")
        field_names = [f.name for f in arcpy.ListFields("sites_sj")]
        print(field_names)
        if "NHDPlusID" in field_names:
            arcpy.CalculateField_management("sites_sj", "catID", "!NHDPlusID!")
        else:
            arcpy.CalculateField_management("sites_sj", "catID", "!gridcode!")
    sites_lst.append(os.path.join(arcpy.env.workspace, "sites_sj"))

#merge all sites_sj into one point file with all the catIDs.
outfile = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\all_sites_catid"
print(sites_lst)
arcpy.Merge_management(sites_lst, outfile)

arr = arcpy.da.FeatureClassToNumPyArray(outfile, ("siteID", "catID"))
print(arr)
sitesDF = pd.DataFrame(arr)


['streams_merge', 'cats_merge', 'region_buf', 'glaciers', 'cats_intersect', 'sites_clip', 'sites_sj']
Feature Class exists
['cats_merge', 'cats_intersect', 'region_buf', 'glaciers', 'sites_clip']
['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'SiteID', 'Wtrbdy_n', 'Agnc_ID', 'SourcNm', 'Cntct_p', 'Cntct_m', 'Cntct_t', 'Snsr_cc', 'file_nm', 'seq_id', 'Snsr_Pl', 'Wtrbdy_t', 'Verified', 'Comments', 'DM_Notes', 'Sn_QAQC', 'HUC8', 'Name', 'NHDPlusID', 'SourceFC', 'GridCode', 'AreaSqKm', 'VPUID', 'HUC8_ID', 'catID']
['streams_merge', 'cats_merge', 'region_buf', 'glaciers', 'cats_intersect', 'sites_clip', 'sites_sj']
Feature Class exists
['W:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\sites_sj', 'W:\\GIS\\AKSSF\\Copper_River\\Copper_River.gdb\\sites_sj', 'W:\\GIS\\AKSSF\\Prince_William_Sound\\Prince_William_Sound.gdb\\sites_sj']
[('kdk_busrv01',  76954) ('kdk_doscr01',  49617) ('kdk_karrv01', 130735)
 ('kdk_doscr02', 106676) ('kdk_olgcr01a',  48267) ('kdk_olgcr01b',  48267)
 ('kdk_ayarv01', 108356) 

9/28/21
STILL WORKING ON THIS, JOIN NOT CORRECT AND NOT SURE WTDS DF HAS EVERYTHING EITHER.

In [27]:
# steps 7-9

import arcpy
import pandas as pd

arcpy.env.workspace = "W:\\GIS\\AKSSF"
print(wtdDf)
print(sitesDF)

#merge sitesDf with wtdDf to get watershed area linked to siteID
join = pd.concat([wtdDf, sitesDF])
print(join)

join.to_csv("W:\\GIS\\AKSSF\\site_wtd_join.csv")


                  Region  catID   Area_km2
0   Prince_William_Sound  18457  [13.1871]
1   Prince_William_Sound  26464   [5.0434]
2   Prince_William_Sound  28086   [4.7622]
3   Prince_William_Sound  29854  [18.7756]
4   Prince_William_Sound  30884    [8.956]
5   Prince_William_Sound  31865   [0.8897]
6   Prince_William_Sound  36645   [4.1284]
7   Prince_William_Sound  37815   [5.8008]
8   Prince_William_Sound  38993   [3.7023]
9   Prince_William_Sound  40285  [12.9112]
10  Prince_William_Sound  41515   [5.8464]
11  Prince_William_Sound  42563  [12.7965]
12  Prince_William_Sound  43055  [15.8122]
13  Prince_William_Sound  43185  [24.4866]
14  Prince_William_Sound  43933  [32.8912]
15  Prince_William_Sound  43973  [17.2326]
16  Prince_William_Sound  44553   [50.826]
17  Prince_William_Sound  44623  [35.4677]
18  Prince_William_Sound  46055   [3.2971]
                         siteID   catID
0                   kdk_busrv01   76954
1                   kdk_doscr01   49617
2                   